In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

from analysis.src.data import Data

In [2]:
data_r = Data(model="resistor", val_cap=None)
data_rcb1 = Data(model="resistor_capacitor_battery", val_cap=1.0)

In [3]:
data_rc10 = Data(model="resistor_capacitor", val_cap=10.0)
data_rc1 = Data(model="resistor_capacitor", val_cap=1.0)
data_rc01 = Data(model="resistor_capacitor", val_cap=0.1)